In [23]:
import tensorflow as tf
import os

In [24]:
os.chdir("E:\Ineuron\Project\Deep Learning Project\DeepCNNClassifier\DEEPCNNCLASSIFIER")

In [25]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [26]:
from dataclasses import dataclass
from pathlib import Path

In [27]:
@dataclass(frozen=True)
class Evaluation_config:
    model_path:Path
    training_data:Path
    params_image_size:list
    params_batch_size:int

In [28]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories,save_json
import tensorflow as tf


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> Evaluation_config:
        return Evaluation_config(model_path=self.config.training.trained_model_path,training_data="artifacts/data_ingestion/PetImages",params_batch_size=self.params.BATCH_SIZE,params_image_size=self.params.IMAGE_SIZE)

In [29]:
import os
import tensorflow as tf
import time
import urllib.request as request
from zipfile import ZipFile
from deepClassifier.entity import TrainingConfig
import tensorflow as tf
from pathlib import Path

class Evaluation:
    def __init__(self, config: Evaluation_config):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(model_path: Path)->tf.keras.Model:
        return tf.keras.models.load_model(model_path)


    def evaluation(self):
        model=self.load_model(self.config.model_path)
        self._valid_generator()
        self.score=model.evaluate(self.valid_generator)
    def save_score(self):
        result={"loss":self.score[0],"accuracy":self.score[1]}
        save_json(Path("scores.json"),data=result)
        
        

        

In [30]:
try:
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    eval=Evaluation(evaluation_config)
    eval.evaluation()
    eval.save_score()
except Exception as e:
    raise e

[2022-09-28 09:27:26,702: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2022-09-28 09:27:26,705: INFO: common]: yaml file: params.yaml loaded successfully
[2022-09-28 09:27:26,707: INFO: common]: created directory at: artifacts
Found 7498 images belonging to 2 classes.
469/469 [==============================] - 1156s 2s/step - loss: 3.2728 - accuracy: 0.7858
[2022-09-28 09:46:43,624: INFO: common]: json file saved at: scores.json
